# Swift Assignment Summary: Transit Performance Analysis

This project analyzes shipment tracking data from a courier logistics network to evaluate transit performance, operational efficiency, and delivery accuracy.
The dataset, provided in nested JSON format, contains detailed shipment and event-level tracking information (including facility movements, timestamps, and delivery milestones).

## Objective

To extract, process, and analyze courier shipment data to compute key logistics performance indicators such as:

Total transit time, facility touchpoints, and inter-facility movement efficiency

Average transit velocity and service-type comparison

Delivery success metrics (first-attempt delivery rates, out-for-delivery attempts)

## Approach

The analysis was performed in six structured phases:

- Data Loading & Exploration:
Loaded and inspected the hierarchical JSON data structure to identify key shipment and event attributes.

- Data Flattening:
Extracted shipment-level and event-level details into a structured tabular DataFrame for analysis.

- Transit Metric Computation:
Calculated per-shipment performance metrics — including total transit hours, facility visits, inter-facility transit time, and delivery attempts.

- Edge Case Handling:
Implemented robust handling for missing fields, inconsistent timestamps ($numberLong vs ISO), null addresses, empty event arrays, and duplicate timestamps.

- Detailed Output Generation:
Exported a clean, shipment-level dataset (transit_performance_detailed.csv) with all computed metrics standardized and timestamped in IST.

- Network Summary Analysis:
Produced an aggregated summary file (transit_performance_summary.csv) capturing overall network trends, facility utilization, service-type comparison, and delivery performance statistics.

## Outcome

The final outputs enable a clear view of courier network performance:

- Average transit time: ~94 hours (≈3.9 days)

- Average facilities per shipment: 2.8

- First-attempt delivery rate: ~85%

- Service analyzed: FedEx Express Saver

This end-to-end analysis provides actionable insights into shipment flow efficiency, delivery accuracy, and potential bottlenecks in transit operations — forming the foundation for data-driven logistics optimization.

## Part 1: Load and Explore Data 
### Objective:
Load the given JSON shipment tracking data, inspect its structure,
and perform basic exploration to understand key fields and hierarchy.


In [31]:
import json
import pandas as pd

# Step 1: Define the path to the dataset

file_path = "C:/Users/Alok verma/Downloads/Swift Assignment 4 - Dataset (2).json"

# Step 2: Load the JSON data

with open(file_path, 'r') as file:
    data = json.load(file)

# Step 3: Quick inspection

print(f"Type of root object: {type(data)}")
print(f"Number of records in JSON: {len(data)}\n")

# Step 4: Explore the first record to understand the structure

first_record = data[0]
print("Keys in first record:\n", first_record.keys())

# Step 5: Drill down to 'trackDetails' — this usually holds shipment details

if 'trackDetails' in first_record:
    print("\nNumber of track details in first record:", len(first_record['trackDetails']))
    if len(first_record['trackDetails']) > 0:
        print("\nKeys inside 'trackDetails[0]':")
        print(first_record['trackDetails'][0].keys())

# Step 6: Explore nested fields within trackDetails

sample_detail = first_record['trackDetails'][0]
print("\nTracking Number:", sample_detail.get('trackingNumber'))
print("Service Info:", sample_detail.get('service', {}))
print("Origin City:", sample_detail.get('shipperAddress', {}).get('city'))
print("Destination City:", sample_detail.get('destinationAddress', {}).get('city'))

# Step 7: Check structure of 'events' array (important for transit performance)

events = sample_detail.get('events', [])
print(f"\nNumber of events: {len(events)}")
if len(events) > 0:
    print("Sample event keys:", events[0].keys())


Type of root object: <class 'list'>
Number of records in JSON: 99

Keys in first record:
 dict_keys(['highestSeverity', 'notifications', 'duplicateWaybill', 'moreData', 'trackDetailsCount', 'trackDetails'])

Number of track details in first record: 1

Keys inside 'trackDetails[0]':
dict_keys(['notification', 'trackingNumber', 'trackingNumberUniqueIdentifier', 'statusDetail', 'informationNotes', 'customerExceptionRequests', 'carrierCode', 'operatingCompanyOrCarrierDescription', 'otherIdentifiers', 'service', 'packageWeight', 'shipmentWeight', 'packaging', 'packageSequenceNumber', 'packageCount', 'shipmentContentPieceCount', 'packageContentPieceCount', 'creatorSoftwareId', 'charges', 'attributes', 'shipmentContents', 'packageContents', 'commodities', 'customsOptionDetails', 'specialHandlings', 'payments', 'shipperAddress', 'datesOrTimes', 'specialInstructions', 'lastUpdatedDestinationAddress', 'destinationAddress', 'actualDeliveryAddress', 'deliveryLocationType', 'deliveryLocationDescrip

## Part 2: Flatten and Extract Transit Data
### Objective:
Extract and flatten shipment-level and event-level details from the nested JSON
structure for each shipment record into a tabular (DataFrame) format.

In [32]:
import pandas as pd
from datetime import datetime

# Define a helper function to safely extract nested fields

def get_value(data_dict, key_path, default=None):
    """Safely extract nested values using a list of keys."""
    current = data_dict
    for key in key_path:
        if isinstance(current, dict) and key in current:
            current = current[key]
        else:
            return default
    return current

# Create an empty list to store flattened records


flattened_records = []

# Iterate through all top-level records
for record in data:
    track_details_list = record.get("trackDetails", [])
    if not track_details_list:
        continue  
    
    # Loop through each shipment inside 'trackDetails'
    
    for shipment in track_details_list:
        tracking_number = shipment.get("trackingNumber")
        service_type = get_value(shipment, ["service", "type"])
        service_description = get_value(shipment, ["service", "description"])
        carrier_code = shipment.get("carrierCode")
        
        # Weight & packaging
        weight_value = get_value(shipment, ["packageWeight", "value"])
        weight_units = get_value(shipment, ["packageWeight", "units"])
        packaging_type = get_value(shipment, ["packaging", "type"])
        
        # Origin (shipper address)
        origin_city = get_value(shipment, ["shipperAddress", "city"])
        origin_state = get_value(shipment, ["shipperAddress", "stateOrProvinceCode"])
        origin_pincode = get_value(shipment, ["shipperAddress", "postalCode"])
        
        # Destination address
        destination_city = get_value(shipment, ["destinationAddress", "city"])
        destination_state = get_value(shipment, ["destinationAddress", "stateOrProvinceCode"])
        destination_pincode = get_value(shipment, ["destinationAddress", "postalCode"])
        
        # Extract events array
        events = shipment.get("events", [])
        
        if not events:
            # Create at least one record even if events missing
            flattened_records.append({
                "tracking_number": tracking_number,
                "service_type": service_type,
                "service_description": service_description,
                "carrier_code": carrier_code,
                "package_weight_value": weight_value,
                "package_weight_units": weight_units,
                "packaging_type": packaging_type,
                "origin_city": origin_city,
                "origin_state": origin_state,
                "origin_pincode": origin_pincode,
                "destination_city": destination_city,
                "destination_state": destination_state,
                "destination_pincode": destination_pincode,
                "event_type": None,
                "event_timestamp": None,
                "event_description": None,
                "event_location_city": None,
                "event_location_state": None,
                "event_location_postal_code": None,
                "arrival_location_type": None
            })
        else:
            # For each event inside shipment
            for event in events:
                # Handle timestamp that could be in $numberLong or ISO format
                raw_timestamp = event.get("timestamp")
                if isinstance(raw_timestamp, dict) and "$numberLong" in raw_timestamp:
                    try:
                        event_timestamp = datetime.fromtimestamp(int(raw_timestamp["$numberLong"]) / 1000)
                    except Exception:
                        event_timestamp = None
                else:
                    event_timestamp = pd.to_datetime(raw_timestamp, errors='coerce')

                flattened_records.append({
                    "tracking_number": tracking_number,
                    "service_type": service_type,
                    "service_description": service_description,
                    "carrier_code": carrier_code,
                    "package_weight_value": weight_value,
                    "package_weight_units": weight_units,
                    "packaging_type": packaging_type,
                    "origin_city": origin_city,
                    "origin_state": origin_state,
                    "origin_pincode": origin_pincode,
                    "destination_city": destination_city,
                    "destination_state": destination_state,
                    "destination_pincode": destination_pincode,
                    "event_type": event.get("eventType"),
                    "event_timestamp": event_timestamp,
                    "event_description": event.get("eventDescription"),
                    "event_location_city": get_value(event, ["address", "city"]),
                    "event_location_state": get_value(event, ["address", "stateOrProvinceCode"]),
                    "event_location_postal_code": get_value(event, ["address", "postalCode"]),
                    "arrival_location_type": event.get("arrivalLocation")
                })

# Convert the flattened list into a DataFrame
df_transit = pd.DataFrame(flattened_records)

# Display basic summary
print(f"Flattening complete! Extracted {len(df_transit)} event records across shipments.\n")
print("Preview of flattened data:")
display(df_transit.head(10))

# Quick info summary
print("\nDataFrame Info:")
print(df_transit.info())


Flattening complete! Extracted 1240 event records across shipments.

Preview of flattened data:


,tracking_number,service_type,service_description,carrier_code,package_weight_value,package_weight_units,packaging_type,origin_city,origin_state,origin_pincode,destination_city,destination_state,destination_pincode,event_type,event_timestamp,event_description,event_location_city,event_location_state,event_location_postal_code,arrival_location_type
0,391128701026,FEDEX_EXPRESS_SAVER,FedEx Economy,FDXE,14.0,KG,YOUR_PACKAGING,Bangalore,KA,None,Gurgaon,HR,None,DL,2020-03-20 13:37:00,Delivered,Gurgaon,HR,122001,DELIVERY_LOCATION
1,391128701026,FEDEX_EXPRESS_SAVER,FedEx Economy,FDXE,14.0,KG,YOUR_PACKAGING,Bangalore,KA,None,Gurgaon,HR,None,OD,2020-03-20 10:16:00,On FedEx vehicle for delivery,MANESAR,HR,122050,VEHICLE
2,391128701026,FEDEX_EXPRESS_SAVER,FedEx Economy,FDXE,14.0,KG,YOUR_PACKAGING,Bangalore,KA,None,Gurgaon,HR,None,IT,2020-03-20 09:18:00,In transit,GURGAON,HR,122001,FEDEX_FACILITY
3,391128701026,FEDEX_EXPRESS_SAVER,FedEx Economy,FDXE,14.0,KG,YOUR_PACKAGING,Bangalore,KA,None,Gurgaon,HR,None,AR,2020-03-20 08:46:00,At local FedEx facility,MANESAR,HR,122050,DESTINATION_FEDEX_FACILITY
4,391128701026,FEDEX_EXPRESS_SAVER,FedEx Economy,FDXE,14.0,KG,YOUR_PACKAGING,Bangalore,KA,None,Gurgaon,HR,None,IT,2020-03-20 01:04:00,In transit,GURGAON,HR,122001,FEDEX_FACILITY
5,391128701026,FEDEX_EXPRESS_SAVER,FedEx Economy,FDXE,14.0,KG,YOUR_PACKAGING,Bangalore,KA,None,Gurgaon,HR,None,IT,2020-03-19 23:15:00,In transit,GURGAON,HR,122001,FEDEX_FACILITY
6,391128701026,FEDEX_EXPRESS_SAVER,FedEx Economy,FDXE,14.0,KG,YOUR_PACKAGING,Bangalore,KA,None,Gurgaon,HR,None,IT,2020-03-17 04:33:00,In transit,BANGALORE,KA,562123,FEDEX_FACILITY
7,391128701026,FEDEX_EXPRESS_SAVER,FedEx Economy,FDXE,14.0,KG,YOUR_PACKAGING,Bangalore,KA,None,Gurgaon,HR,None,IT,2020-03-17 00:06:00,In transit,BANGALORE,KA,562123,FEDEX_FACILITY
8,391128701026,FEDEX_EXPRESS_SAVER,FedEx Economy,FDXE,14.0,KG,YOUR_PACKAGING,Bangalore,KA,None,Gurgaon,HR,None,DP,2020-03-16 22:54:00,Left FedEx origin facility,BANGALORE,KA,560048,ORIGIN_FEDEX_FACILITY
9,391128701026,FEDEX_EXPRESS_SAVER,FedEx Economy,FDXE,14.0,KG,YOUR_PACKAGING,Bangalore,KA,None,Gurgaon,HR,None,PU,2020-03-16 15:44:00,Picked up,BANGALORE,KA,560048,PICKUP_LOCATION



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240 entries, 0 to 1239
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   tracking_number             1240 non-null   object        
 1   service_type                1240 non-null   object        
 2   service_description         1240 non-null   object        
 3   carrier_code                1240 non-null   object        
 4   package_weight_value        1240 non-null   float64       
 5   package_weight_units        1240 non-null   object        
 6   packaging_type              1240 non-null   object        
 7   origin_city                 1240 non-null   object        
 8   origin_state                1240 non-null   object        
 9   origin_pincode              0 non-null      object        
 10  destination_city            1240 non-null   object        
 11  destination_state           1240 non-nu

## Part 3: Compute Transit Performance Metrics

Preconditions:
- `data` : the loaded JSON list (root)
- `df_transit` : flattened DataFrame from Part 2 (one row per event)
If you don't have df_transit, re-run Part 2 first.

In [33]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone


def parse_timestamp(ts):
    """
    Parse different timestamp formats:
     - dict with "$numberLong" (milliseconds since epoch)
     - ISO string or pandas-parsable string
     - datetime objects pass-through
    Returns a pandas.Timestamp or NaT.
    """
    if ts is None:
        return pd.NaT
    # already pandas Timestamp or datetime
    if isinstance(ts, (pd.Timestamp, datetime)):
        return pd.to_datetime(ts)
    # dict with $numberLong
    if isinstance(ts, dict) and "$numberLong" in ts:
        try:
            ms = int(ts["$numberLong"])
            return pd.to_datetime(ms, unit='ms', utc=True).tz_convert('Asia/Kolkata')
        except Exception:
            return pd.NaT
    # plain numeric string/number (epoch ms or s) - try safe conversions
    if isinstance(ts, (int, float)) or (isinstance(ts, str) and ts.isdigit()):
        try:
            val = int(ts)
            if val > 1e12: 
                return pd.to_datetime(val, unit='ms', utc=True).tz_convert('Asia/Kolkata')
            else:
                return pd.to_datetime(val, unit='s', utc=True).tz_convert('Asia/Kolkata')
        except Exception:
            pass
        
    # fallback to pandas parsing (assume timezone info included where present)
    try:
        parsed = pd.to_datetime(ts, errors='coerce')
        if pd.isna(parsed):
            return pd.NaT
        # if timezone-naive, localize to Asia/Kolkata (IST) for consistent calculations
        if parsed.tzinfo is None:
            # treat as already in IST if string had +05:30 or else assume IST
            try:
                return parsed.tz_localize('Asia/Kolkata')
            except Exception:
                return parsed.tz_convert('Asia/Kolkata') if parsed.tzinfo else parsed
        else:
            # convert to IST
            return parsed.tz_convert('Asia/Kolkata')
    except Exception:
        return pd.NaT

def safe_lower(x):
    return x.lower() if isinstance(x, str) else ""

def contains_any(text, keywords):
    txt = safe_lower(text)
    return any(k in txt for k in keywords)

# Event classification keyword lists (heuristic)
IN_TRANSIT_KEYWORDS = ["in transit", "on vehicle", "on fedex vehicle", "departed", "left facility", "transit", "on truck", "in route"]
ARRIVAL_KEYWORDS = ["arrived", "arrival", "arrive at", "arrived at", "at facility", "arrival scan"]
OUT_FOR_DELIVERY_KEYWORDS = ["out for delivery", "on vehicle for delivery", "on fedex vehicle for delivery", "out for delivery -", "on truck for delivery"]

# Decide event-type driven buckets (we will also use textual fallback)
# These sets can be extended if you discover more codes in your dataset

PICKUP_EVENT_TYPES = set(["PU", "PUP", "PICKUP"])   # common pickup codes
DELIVERY_EVENT_TYPES = set(["DL", "DE", "DEL"])     # delivery / delivered
OUT_FOR_DELIVERY_TYPES = set(["OD", "OF"])          # often OD (on delivery / out for delivery)
FACILITY_SUBSTRING = "FACILITY"

# -------------------------
# Precompute: per-shipment grouping
# -------------------------
# get list of unique tracking numbers present in df_transit or raw 

if 'df_transit' not in globals():
    # build df_transit quickly as fallback (in case user didn't run Part 2)
    # minimal reconstruction: iterate raw data similar to Part 2
    flattened_records = []
    def get_val(d, path, default=None):
        cur = d
        for k in path:
            if isinstance(cur, dict) and k in cur:
                cur = cur[k]
            else:
                return default
        return cur
    for rec in data:
        for shipment in rec.get("trackDetails", []):
            events = shipment.get("events", []) or []
            # if no events, still create one row
            if not events:
                flattened_records.append({
                    "tracking_number": shipment.get("trackingNumber"),
                    "service_type": get_val(shipment, ["service", "type"]),
                    "service_description": get_val(shipment, ["service", "description"]),
                    "carrier_code": shipment.get("carrierCode"),
                    "package_weight_value": get_val(shipment, ["packageWeight", "value"]),
                    "package_weight_units": get_val(shipment, ["packageWeight", "units"]),
                    "packaging_type": get_val(shipment, ["packaging", "type"]),
                    "origin_city": get_val(shipment, ["shipperAddress","city"]),
                    "origin_state": get_val(shipment, ["shipperAddress","stateOrProvinceCode"]),
                    "origin_pincode": get_val(shipment, ["shipperAddress","postalCode"]),
                    "destination_city": get_val(shipment, ["destinationAddress","city"]),
                    "destination_state": get_val(shipment, ["destinationAddress","stateOrProvinceCode"]),
                    "destination_pincode": get_val(shipment, ["destinationAddress","postalCode"]),
                    "event_type": None,
                    "event_timestamp": pd.NaT,
                    "event_description": None,
                    "event_location_city": None,
                    "event_location_state": None,
                    "event_location_postal_code": None,
                    "arrival_location_type": None,
                    "raw_shipment_obj": shipment
                })
            else:
                for ev in events:
                    flattened_records.append({
                        "tracking_number": shipment.get("trackingNumber"),
                        "service_type": get_val(shipment, ["service", "type"]),
                        "service_description": get_val(shipment, ["service", "description"]),
                        "carrier_code": shipment.get("carrierCode"),
                        "package_weight_value": get_val(shipment, ["packageWeight", "value"]),
                        "package_weight_units": get_val(shipment, ["packageWeight", "units"]),
                        "packaging_type": get_val(shipment, ["packaging", "type"]),
                        "origin_city": get_val(shipment, ["shipperAddress","city"]),
                        "origin_state": get_val(shipment, ["shipperAddress","stateOrProvinceCode"]),
                        "origin_pincode": get_val(shipment, ["shipperAddress","postalCode"]),
                        "destination_city": get_val(shipment, ["destinationAddress","city"]),
                        "destination_state": get_val(shipment, ["destinationAddress","stateOrProvinceCode"]),
                        "destination_pincode": get_val(shipment, ["destinationAddress","postalCode"]),
                        "event_type": ev.get("eventType"),
                        "event_timestamp": ev.get("timestamp"),
                        "event_description": ev.get("eventDescription"),
                        "event_location_city": (ev.get("address") or {}).get("city"),
                        "event_location_state": (ev.get("address") or {}).get("stateOrProvinceCode"),
                        "event_location_postal_code": (ev.get("address") or {}).get("postalCode"),
                        "arrival_location_type": ev.get("arrivalLocation"),
                        "raw_shipment_obj": shipment
                    })
    df_transit = pd.DataFrame(flattened_records)

# Ensure event_timestamp parsed consistently

if 'event_timestamp' in df_transit.columns:
    df_transit['event_timestamp_parsed'] = df_transit['event_timestamp'].apply(parse_timestamp)

# Build mapping from tracking_number -> original shipment object (for datesOrTimes and deliveryLocationType)

shipment_lookup = {}
for rec in data:
    for shp in rec.get("trackDetails", []):
        tn = shp.get("trackingNumber")
        if tn:
            shipment_lookup[tn] = shp

unique_event_types = sorted(df_transit['event_type'].dropna().unique().tolist())
print("Unique event types in dataset:", unique_event_types)

metrics = []
grouped = df_transit.groupby('tracking_number', dropna=False)

for tracking_number, group in grouped:
    service_type = group['service_type'].dropna().iloc[0] if group['service_type'].dropna().any() else None
    carrier_code = group['carrier_code'].dropna().iloc[0] if group['carrier_code'].dropna().any() else None
    delivery_location_type = None
    shp_obj = shipment_lookup.get(tracking_number)
    if shp_obj:
        delivery_location_type = shp_obj.get('deliveryLocationType')
    
    group = group.copy()
    if 'event_timestamp_parsed' not in group.columns:
        group['event_timestamp_parsed'] = group['event_timestamp'].apply(parse_timestamp)
    group = group.sort_values(by='event_timestamp_parsed', na_position='last').reset_index(drop=True)
    
    # Unique facility touchpoints: distinct arrivalLocation values containing "FACILITY"
    # also consider event-level arrival_location_type column if present
    
    facility_events = group[
        group['arrival_location_type'].astype(str).str.contains(FACILITY_SUBSTRING, case=False, na=False)
    ].copy()
    unique_facilities = facility_events['arrival_location_type'].dropna().unique().tolist()
    num_unique_facilities = len(unique_facilities)

    event_type_counts = group['event_type'].fillna('UNKNOWN').value_counts().to_dict()

    def is_in_transit_row(row):
        et = safe_lower(row.get('event_type') or "")
        desc = row.get('event_description') or ""
        # check explicit type codes
        if row.get('event_type') in IN_TRANSIT_KEYWORDS:  # rare - leave for completeness
            return True
        # if eventType is one of typical in-transit codes
        if row.get('event_type') in ("IT", "TR", "DP", "OC", "OD"):  # 'OD' sometimes means out for delivery though
            return True
        # textual check
        if contains_any(desc, IN_TRANSIT_KEYWORDS):
            return True
        return False
    
    num_in_transit_events = group.apply(is_in_transit_row, axis=1).sum()
    
    # Identify pickup and delivery times (best-effort)
    pickup_ts = pd.NaT
    delivery_ts = pd.NaT
    
    # 1) Prefer datesOrTimes entries on shipment (ACTUAL_PICKUP, ACTUAL_TENDER for pickup; ACTUAL_DELIVERY for delivery)
    if shp_obj:
        for dt in shp_obj.get('datesOrTimes', []) or []:
            try:
                dtype = dt.get('type', '').upper()
                raw = dt.get('dateOrTimestamp')
                parsed = parse_timestamp(raw)
                if dtype == 'ACTUAL_PICKUP' or dtype == 'ACTUAL_TENDER':
                    # choose earliest of these if multiple
                    if pd.isna(pickup_ts) or parsed < pickup_ts:
                        pickup_ts = parsed
                elif dtype == 'ACTUAL_DELIVERY':
                    if pd.isna(delivery_ts) or parsed > delivery_ts:
                        delivery_ts = parsed
            except Exception:
                continue
    
    # 2) Fallback to events: pickup = first event that looks like a pickup; delivery = last that looks like delivery
    # Find pickup in events
    if pd.isna(pickup_ts):
        # scan events in chronological order
        for _, row in group.iterrows():
            desc = row.get('event_description') or ""
            etype = (row.get('event_type') or "").upper()
            ts = row.get('event_timestamp_parsed')
            # check eventType tokens and description
            if etype in PICKUP_EVENT_TYPES or contains_any(desc, ["pickup", "picked up", "tendered"]):
                pickup_ts = ts
                break
        if pd.isna(pickup_ts) and group['event_timestamp_parsed'].notna().any():
            pickup_ts = group['event_timestamp_parsed'].min()
    
    if pd.isna(delivery_ts):
        # reverse chronological scan to find "delivered" or DL
        for _, row in group.iloc[::-1].iterrows():
            desc = row.get('event_description') or ""
            etype = (row.get('event_type') or "").upper()
            ts = row.get('event_timestamp_parsed')
            if etype in DELIVERY_EVENT_TYPES or contains_any(desc, ["delivered", "delivery"]):
                delivery_ts = ts
                break
        # fallback: latest event timestamp
        if pd.isna(delivery_ts) and group['event_timestamp_parsed'].notna().any():
            delivery_ts = group['event_timestamp_parsed'].max()
    
    # Compute total transit hours (pickup -> delivery)
    total_transit_hours = np.nan
    if pd.notna(pickup_ts) and pd.notna(delivery_ts):
        # ensure both in same tz and compute hours
        try:
            delta = delivery_ts - pickup_ts
            total_transit_hours = delta.total_seconds() / 3600.0
            # if negative (bad data), set to NaN
            if total_transit_hours < 0:
                total_transit_hours = np.nan
        except Exception:
            total_transit_hours = np.nan
    
    # Inter-facility transit time:
    # We'll compute time between successive facility event timestamps (i.e., travel time between facilities)
    # Sum of (timestamp_next_facility - timestamp_prev_facility)
    time_in_inter_facility_hours = np.nan
    if len(facility_events) >= 2:
        # ensure parsed timestamps for facility_events
        fac_ts = facility_events['event_timestamp_parsed'].dropna().sort_values()
        if len(fac_ts) >= 2:
            diffs = fac_ts.diff().dropna().dt.total_seconds() / 3600.0
            # sum positive diffs (ignore negative/nonsensical)
            positive_sum = diffs[diffs > 0].sum()
            time_in_inter_facility_hours = float(positive_sum)
        else:
            time_in_inter_facility_hours = 0.0
    elif len(facility_events) == 1:
        # only one facility touchpoint — inter-facility transit undefined; set 0.0
        time_in_inter_facility_hours = 0.0
    else:
        # no facility events found
        time_in_inter_facility_hours = 0.0
    
    # Transit velocity: avg hours per facility
    avg_hours_per_facility = np.nan
    if num_unique_facilities > 0 and pd.notna(total_transit_hours):
        avg_hours_per_facility = total_transit_hours / num_unique_facilities
    elif num_unique_facilities > 0 and pd.isna(total_transit_hours):
        avg_hours_per_facility = np.nan
    else:
        avg_hours_per_facility = np.nan
    
    # Service category classification (simple heuristics)
    svc = service_type or ""
    svc_upper = svc.upper() if isinstance(svc, str) else ""
    if "EXPRESS" in svc_upper or "XS" in svc_upper or "FEDEX_EXPRESS" in svc_upper or "PRIORITY" in svc_upper:
        is_express_service = True
    elif "ECONOMY" in svc_upper or "STANDARD" in svc_upper or "GROUND" in svc_upper:
        is_express_service = False
    else:
        # Unknown -> treat as False but keep as 'unknown' if needed
        is_express_service = False
    
    # Delivery Characteristics: out-for-delivery attempts
    def is_out_for_delivery_row(row):
        et = (row.get('event_type') or "").upper()
        desc = row.get('event_description') or ""
        if et in OUT_FOR_DELIVERY_TYPES or 'OD' == et or contains_any(desc, OUT_FOR_DELIVERY_KEYWORDS):
            return True
        return False
    
    num_out_for_delivery_attempts = group.apply(is_out_for_delivery_row, axis=1).sum()
    first_attempt_delivery = None
    
    # if delivered and number of out-for-delivery attempts == 1 then True; if 0 and delivered maybe direct delivered; treat as True
    # We'll inspect whether a delivery event exists
    
    delivered_exists = group.apply(lambda r: ((r.get('event_type') or "").upper() in DELIVERY_EVENT_TYPES) or contains_any(r.get('event_description') or "", ["delivered"]), axis=1).any()
    if delivered_exists:
        # Consider TRUE if <=1 OFD events
        first_attempt_delivery = (num_out_for_delivery_attempts <= 1)
    else:
        first_attempt_delivery = False  # not delivered yet
    
    # total events count
    total_events_count = len(group)
    
    # assemble metrics
    metrics.append({
        "tracking_number": tracking_number,
        "service_type": service_type,
        "carrier_code": carrier_code,
        "package_weight_value": group['package_weight_value'].dropna().iloc[0] if group['package_weight_value'].dropna().any() else None,
        "package_weight_units": group['package_weight_units'].dropna().iloc[0] if group['package_weight_units'].dropna().any() else None,
        "origin_city": group['origin_city'].dropna().iloc[0] if group['origin_city'].dropna().any() else None,
        "destination_city": group['destination_city'].dropna().iloc[0] if group['destination_city'].dropna().any() else None,
        "pickup_datetime_ist": pickup_ts,
        "delivery_datetime_ist": delivery_ts,
        "total_transit_hours": total_transit_hours,
        "num_facilities_visited": num_unique_facilities,
        "num_in_transit_events": int(num_in_transit_events),
        "time_in_inter_facility_transit_hours": time_in_inter_facility_hours,
        "avg_hours_per_facility": avg_hours_per_facility,
        "is_express_service": bool(is_express_service),
        "delivery_location_type": delivery_location_type,
        "num_out_for_delivery_attempts": int(num_out_for_delivery_attempts),
        "first_attempt_delivery": bool(first_attempt_delivery),
        "total_events_count": int(total_events_count),
        "event_type_counts": event_type_counts
    })

# Final metrics DataFrame
df_metrics = pd.DataFrame(metrics)

# Ensure timestamp columns are pandas datetime and in IST tz
if 'pickup_datetime_ist' in df_metrics.columns:
    df_metrics['pickup_datetime_ist'] = pd.to_datetime(df_metrics['pickup_datetime_ist'])
    # convert to tz 'Asia/Kolkata' if naive
    df_metrics['pickup_datetime_ist'] = df_metrics['pickup_datetime_ist'].apply(lambda x: x.tz_convert('Asia/Kolkata') if hasattr(x, 'tzinfo') and x.tzinfo else (x.tz_localize('Asia/Kolkata') if pd.notna(x) else x))
if 'delivery_datetime_ist' in df_metrics.columns:
    df_metrics['delivery_datetime_ist'] = pd.to_datetime(df_metrics['delivery_datetime_ist'])
    df_metrics['delivery_datetime_ist'] = df_metrics['delivery_datetime_ist'].apply(lambda x: x.tz_convert('Asia/Kolkata') if hasattr(x, 'tzinfo') and x.tzinfo else (x.tz_localize('Asia/Kolkata') if pd.notna(x) else x))

# Quick sanity checks / display
print("Computation complete. Per-shipment metrics generated for", len(df_metrics), "shipments.\n")
display(df_metrics.head(10))

# Also expose the global unique_event_types which may be useful downstream
print("\nUnique event types found (global):", unique_event_types)


Unique event types in dataset: ['AF', 'AR', 'AS', 'DE', 'DL', 'DP', 'IT', 'OC', 'OD', 'PU', 'SE']
Computation complete. Per-shipment metrics generated for 99 shipments.



,tracking_number,service_type,carrier_code,package_weight_value,package_weight_units,origin_city,destination_city,pickup_datetime_ist,delivery_datetime_ist,total_transit_hours,num_facilities_visited,num_in_transit_events,time_in_inter_facility_transit_hours,avg_hours_per_facility,is_express_service,delivery_location_type,num_out_for_delivery_attempts,first_attempt_delivery,total_events_count,event_type_counts
0,280267328981,FEDEX_EXPRESS_SAVER,FDXE,20.0,KG,Delhi,Hyderabad,2021-06-11 18:56:00+05:30,2021-06-18 17:18:00+05:30,166.366667,3,11,155.633333,55.455556,True,IN_BOND_OR_CAGE,0,True,16,"{'IT': 9, 'AR': 2, 'OC': 1, 'PU': 1, 'DP': 1, ..."
1,280267329094,FEDEX_EXPRESS_SAVER,FDXE,20.0,KG,Delhi,Noida,2021-06-11 18:56:00+05:30,2021-06-16 11:28:00+05:30,112.533333,3,9,107.550000,37.511111,True,RECEPTIONIST_OR_FRONT_DESK,0,True,18,"{'IT': 7, 'AR': 4, 'DE': 2, 'OC': 1, 'PU': 1, ..."
2,280307632740,FEDEX_EXPRESS_SAVER,FDXE,2.0,KG,Mumbai,Pune,2021-06-14 19:31:00+05:30,2021-06-16 16:30:00+05:30,44.983333,3,7,38.133333,14.994444,True,RESIDENCE,1,True,11,"{'IT': 4, 'OC': 1, 'PU': 1, 'DP': 1, 'AF': 1, ..."
3,280307633276,FEDEX_EXPRESS_SAVER,FDXE,2.0,KG,Mumbai,Pune,2021-06-14 19:31:00+05:30,2021-06-16 16:30:00+05:30,44.983333,3,7,38.166667,14.994444,True,RESIDENCE,1,True,11,"{'IT': 4, 'OC': 1, 'PU': 1, 'DP': 1, 'AF': 1, ..."
4,280439181099,FEDEX_EXPRESS_SAVER,FDXE,32.0,KG,Delhi,Chennai,2021-06-16 19:22:00+05:30,2021-06-24 18:22:00+05:30,191.000000,3,9,180.150000,63.666667,True,RECEPTIONIST_OR_FRONT_DESK,1,True,17,"{'IT': 6, 'AF': 3, 'AR': 2, 'OC': 1, 'PU': 1, ..."
5,280853182067,FEDEX_EXPRESS_SAVER,FDXE,5.0,KG,Bangalore,Chennai,2021-06-28 18:18:00+05:30,2021-07-02 19:19:00+05:30,97.016667,3,8,70.566667,32.338889,True,RESIDENCE,2,False,14,"{'IT': 4, 'AR': 3, 'OD': 2, 'OC': 1, 'PU': 1, ..."
6,280902855329,FEDEX_EXPRESS_SAVER,FDXE,2.0,KG,Bangalore,Delhi,2021-06-29 19:07:00+05:30,2021-07-05 15:57:00+05:30,140.833333,3,11,87.250000,46.944444,True,RESIDENCE,2,False,17,"{'IT': 6, 'DP': 2, 'AR': 2, 'DE': 2, 'OD': 2, ..."
7,280902966660,FEDEX_EXPRESS_SAVER,FDXE,2.5,KG,Bangalore,Hyderabad,2021-06-29 19:06:00+05:30,2021-07-02 19:59:00+05:30,72.883333,3,7,62.266667,24.294444,True,IN_BOND_OR_CAGE,0,True,12,"{'IT': 5, 'AR': 2, 'OC': 1, 'PU': 1, 'DP': 1, ..."
8,280993568461,FEDEX_EXPRESS_SAVER,FDXE,1.5,KG,Pune,Mumbai,2021-07-03 16:19:00+05:30,2021-07-06 10:30:00+05:30,66.183333,3,7,41.383333,22.061111,True,RESIDENCE,2,False,12,"{'IT': 3, 'AR': 2, 'OD': 2, 'OC': 1, 'PU': 1, ..."
9,280998636780,FEDEX_EXPRESS_SAVER,FDXE,3.8,KG,Mumbai,Surat,2021-07-03 18:10:00+05:30,2021-07-07 17:34:00+05:30,95.400000,3,11,70.350000,31.800000,True,RESIDENCE,2,False,16,"{'IT': 7, 'AR': 2, 'OD': 2, 'OC': 1, 'PU': 1, ..."



Unique event types found (global): ['AF', 'AR', 'AS', 'DE', 'DL', 'DP', 'IT', 'OC', 'OD', 'PU', 'SE']


## Part 4: Handle Edge Cases
### Objective:
Clean and validate the data produced so far (df_transit and df_metrics)
to ensure robustness against:
- Missing/null fields
- Timestamps in mixed formats
- Incomplete event sequences
- Missing address information
- Duplicate events
- Empty or missing events arrays

In [34]:
import numpy as np
import pandas as pd

def safe_parse_timestamp(ts):
    """Wrapper to re-parse timestamps robustly; fallbacks included."""
    try:
        if isinstance(ts, pd.Timestamp):
            return ts
        if isinstance(ts, dict) and "$numberLong" in ts:
            return pd.to_datetime(int(ts["$numberLong"]), unit="ms", errors="coerce")
        return pd.to_datetime(ts, errors="coerce")
    except Exception:
        return pd.NaT

def fill_missing_str(series, fill="UNKNOWN"):
    """Replace missing string values with a safe placeholder."""
    return series.fillna(fill).replace("", fill)

print("Cleaning missing or null values...")

string_cols = [
    "tracking_number","service_type","service_description","carrier_code",
    "packaging_type","origin_city","origin_state","origin_pincode",
    "destination_city","destination_state","destination_pincode",
    "event_type","event_description","event_location_city",
    "event_location_state","event_location_postal_code","arrival_location_type"
]
for col in string_cols:
    if col in df_transit.columns:
        df_transit[col] = fill_missing_str(df_transit[col])

# Replace NaN numeric fields with 0 or a sentinel
for col in ["package_weight_value"]:
    if col in df_transit.columns:
        df_transit[col] = pd.to_numeric(df_transit[col], errors="coerce").fillna(0)

print("Normalizing timestamp formats...")

if "event_timestamp" in df_transit.columns:
    df_transit["event_timestamp_parsed"] = df_transit["event_timestamp"].apply(safe_parse_timestamp)
    df_transit["event_timestamp_parsed"] = (
        df_transit["event_timestamp_parsed"]
        .apply(lambda x: x.tz_localize("Asia/Kolkata") if pd.notna(x) and x.tzinfo is None else x)
        .apply(lambda x: x.tz_convert("Asia/Kolkata") if pd.notna(x) else x)
    )

print("Removing duplicate events (same tracking number + timestamp)...")

df_transit = df_transit.sort_values(["tracking_number","event_timestamp_parsed"])
df_transit = df_transit.drop_duplicates(subset=["tracking_number","event_timestamp_parsed"], keep="first")

print("Handling missing or empty events arrays...")

# For shipments with no events, we already created placeholder rows in Part 2.
# Reconfirm at least one row per tracking number.
all_tracking_numbers = set([shp.get("trackingNumber") for rec in data for shp in rec.get("trackDetails", [])])
missing_shipments = all_tracking_numbers - set(df_transit["tracking_number"].unique())

if missing_shipments:
    placeholder_rows = []
    for tn in missing_shipments:
        shp_obj = shipment_lookup.get(tn, {})
        placeholder_rows.append({
            "tracking_number": tn,
            "service_type": (shp_obj.get("service") or {}).get("type","UNKNOWN"),
            "service_description": (shp_obj.get("service") or {}).get("description","UNKNOWN"),
            "carrier_code": shp_obj.get("carrierCode","UNKNOWN"),
            "package_weight_value": (shp_obj.get("packageWeight") or {}).get("value",0),
            "package_weight_units": (shp_obj.get("packageWeight") or {}).get("units","KG"),
            "packaging_type": (shp_obj.get("packaging") or {}).get("type","UNKNOWN"),
            "origin_city": (shp_obj.get("shipperAddress") or {}).get("city","UNKNOWN"),
            "origin_state": (shp_obj.get("shipperAddress") or {}).get("stateOrProvinceCode","UNKNOWN"),
            "origin_pincode": (shp_obj.get("shipperAddress") or {}).get("postalCode","UNKNOWN"),
            "destination_city": (shp_obj.get("destinationAddress") or {}).get("city","UNKNOWN"),
            "destination_state": (shp_obj.get("destinationAddress") or {}).get("stateOrProvinceCode","UNKNOWN"),
            "destination_pincode": (shp_obj.get("destinationAddress") or {}).get("postalCode","UNKNOWN"),
            "event_type": "NO_EVENT",
            "event_timestamp_parsed": pd.NaT,
            "event_description": "No events recorded",
            "event_location_city": "UNKNOWN",
            "event_location_state": "UNKNOWN",
            "event_location_postal_code": "UNKNOWN",
            "arrival_location_type": "NONE"
        })
    df_transit = pd.concat([df_transit, pd.DataFrame(placeholder_rows)], ignore_index=True)

# ---------- Handle incomplete event sequences ----------
print("Checking for incomplete event sequences...")

# Mark shipments missing pickup/delivery as incomplete
df_metrics["is_incomplete_sequence"] = df_metrics.apply(
    lambda x: pd.isna(x["pickup_datetime_ist"]) or pd.isna(x["delivery_datetime_ist"]), axis=1
)

# Fill missing times with NaT-safe fallback
df_metrics["pickup_datetime_ist"] = pd.to_datetime(df_metrics["pickup_datetime_ist"], errors="coerce")
df_metrics["delivery_datetime_ist"] = pd.to_datetime(df_metrics["delivery_datetime_ist"], errors="coerce")

# If pickup or delivery missing, set total_transit_hours to NaN
df_metrics.loc[df_metrics["is_incomplete_sequence"], "total_transit_hours"] = np.nan

print("Replacing missing address fields...")

for col in ["origin_city","destination_city","origin_state","destination_state"]:
    if col in df_metrics.columns:
        df_metrics[col] = fill_missing_str(df_metrics[col])

print("Edge case handling complete!")

print(f"\nTotal shipments (unique): {df_metrics['tracking_number'].nunique()}")
print(f"Total events (after cleaning): {len(df_transit)}")
print(f"Incomplete shipments flagged: {df_metrics['is_incomplete_sequence'].sum()}")

display(df_metrics.head(5))


Cleaning missing or null values...
Normalizing timestamp formats...
Removing duplicate events (same tracking number + timestamp)...
Handling missing or empty events arrays...
Checking for incomplete event sequences...
Replacing missing address fields...
Edge case handling complete!

Total shipments (unique): 99
Total events (after cleaning): 1196
Incomplete shipments flagged: 0


,tracking_number,service_type,carrier_code,package_weight_value,package_weight_units,origin_city,destination_city,pickup_datetime_ist,delivery_datetime_ist,total_transit_hours,...,num_in_transit_events,time_in_inter_facility_transit_hours,avg_hours_per_facility,is_express_service,delivery_location_type,num_out_for_delivery_attempts,first_attempt_delivery,total_events_count,event_type_counts,is_incomplete_sequence
0,280267328981,FEDEX_EXPRESS_SAVER,FDXE,20.0,KG,Delhi,Hyderabad,2021-06-11 18:56:00+05:30,2021-06-18 17:18:00+05:30,166.366667,...,11,155.633333,55.455556,True,IN_BOND_OR_CAGE,0,True,16,"{'IT': 9, 'AR': 2, 'OC': 1, 'PU': 1, 'DP': 1, ...",False
1,280267329094,FEDEX_EXPRESS_SAVER,FDXE,20.0,KG,Delhi,Noida,2021-06-11 18:56:00+05:30,2021-06-16 11:28:00+05:30,112.533333,...,9,107.550000,37.511111,True,RECEPTIONIST_OR_FRONT_DESK,0,True,18,"{'IT': 7, 'AR': 4, 'DE': 2, 'OC': 1, 'PU': 1, ...",False
2,280307632740,FEDEX_EXPRESS_SAVER,FDXE,2.0,KG,Mumbai,Pune,2021-06-14 19:31:00+05:30,2021-06-16 16:30:00+05:30,44.983333,...,7,38.133333,14.994444,True,RESIDENCE,1,True,11,"{'IT': 4, 'OC': 1, 'PU': 1, 'DP': 1, 'AF': 1, ...",False
3,280307633276,FEDEX_EXPRESS_SAVER,FDXE,2.0,KG,Mumbai,Pune,2021-06-14 19:31:00+05:30,2021-06-16 16:30:00+05:30,44.983333,...,7,38.166667,14.994444,True,RESIDENCE,1,True,11,"{'IT': 4, 'OC': 1, 'PU': 1, 'DP': 1, 'AF': 1, ...",False
4,280439181099,FEDEX_EXPRESS_SAVER,FDXE,32.0,KG,Delhi,Chennai,2021-06-16 19:22:00+05:30,2021-06-24 18:22:00+05:30,191.000000,...,9,180.150000,63.666667,True,RECEPTIONIST_OR_FRONT_DESK,1,True,17,"{'IT': 6, 'AF': 3, 'AR': 2, 'OC': 1, 'PU': 1, ...",False


## Part 5 (Fixed): Output Detailed Transit CSV

### Objective:
Create transit_performance_detailed.csv with all required columns.
Automatically handles missing columns safely.

In [35]:
import os
import numpy as np
import pandas as pd

# Ensure all required columns exist
required_cols = [
    "tracking_number",
    "service_type",
    "carrier_code",
    "package_weight_value",
    "package_weight_units",
    "packaging_type",
    "origin_city",
    "origin_state",
    "origin_pincode",
    "destination_city",
    "destination_state",
    "destination_pincode",
    "pickup_datetime_ist",
    "delivery_datetime_ist",
    "total_transit_hours",
    "num_facilities_visited",
    "num_in_transit_events",
    "time_in_inter_facility_transit_hours",
    "avg_hours_per_facility",
    "is_express_service",
    "delivery_location_type",
    "num_out_for_delivery_attempts",
    "first_attempt_delivery",
    "total_events_count"
]

# Add missing columns with default placeholders

for col in required_cols:
    if col not in df_metrics.columns:
        if "city" in col or "state" in col or "pincode" in col or "packaging" in col:
            df_metrics[col] = "UNKNOWN"
        elif "num_" in col or "hours" in col or "weight" in col:
            df_metrics[col] = 0
        elif "is_express" in col or "first_attempt" in col:
            df_metrics[col] = False
        elif "datetime" in col:
            df_metrics[col] = pd.NaT
        else:
            df_metrics[col] = np.nan

def convert_to_kg(value, unit):
    """Convert weight to kilograms if needed."""
    try:
        if pd.isna(value):
            return 0.0
        if isinstance(value, str):
            value = float(value)
    except Exception:
        return 0.0
    
    unit = (unit or "").upper()
    if unit in ["KG", "KGS", "KILOGRAM", "KILOGRAMS"]:
        return round(value, 3)
    elif unit in ["LB", "LBS", "POUND", "POUNDS"]:
        return round(value * 0.453592, 3)
    else:
        return round(value, 3)  

df_metrics["package_weight_kg"] = df_metrics.apply(
    lambda x: convert_to_kg(x.get("package_weight_value"), x.get("package_weight_units")), axis=1
)

def format_ts(ts):
    if pd.isna(ts):
        return ""
    try:
        ts = pd.to_datetime(ts, errors="coerce")
        if pd.isna(ts):
            return ""
        if ts.tzinfo is None:
            ts = ts.tz_localize("Asia/Kolkata")
        else:
            ts = ts.tz_convert("Asia/Kolkata")
        return ts.strftime("%Y-%m-%d %H:%M:%S %Z")
    except Exception:
        return ""

df_metrics["pickup_datetime_ist_str"] = df_metrics["pickup_datetime_ist"].apply(format_ts)
df_metrics["delivery_datetime_ist_str"] = df_metrics["delivery_datetime_ist"].apply(format_ts)

final_cols = [
    "tracking_number",
    "service_type",
    "carrier_code",
    "package_weight_kg",
    "packaging_type",
    "origin_city",
    "origin_state",
    "origin_pincode",
    "destination_city",
    "destination_state",
    "destination_pincode",
    "pickup_datetime_ist_str",
    "delivery_datetime_ist_str",
    "total_transit_hours",
    "num_facilities_visited",
    "num_in_transit_events",
    "time_in_inter_facility_transit_hours",
    "avg_hours_per_facility",
    "is_express_service",
    "delivery_location_type",
    "num_out_for_delivery_attempts",
    "first_attempt_delivery",
    "total_events_count"
]

rename_map = {
    "pickup_datetime_ist_str": "pickup_datetime_ist",
    "delivery_datetime_ist_str": "delivery_datetime_ist"
}

df_detailed_export = df_metrics[final_cols].rename(columns=rename_map)

output_filename = "transit_performance_detailed.csv"
df_detailed_export.to_csv(output_filename, index=False, encoding="utf-8")

print(f"Detailed transit performance CSV created successfully!")
print(f"File Path: {os.path.abspath(output_filename)}")
print(f"Total Shipments Exported: {len(df_detailed_export)}")

display(df_detailed_export.head(10))


Detailed transit performance CSV created successfully!
File Path: C:\Users\Alok verma\transit_performance_detailed.csv
Total Shipments Exported: 99


,tracking_number,service_type,carrier_code,package_weight_kg,packaging_type,origin_city,origin_state,origin_pincode,destination_city,destination_state,...,total_transit_hours,num_facilities_visited,num_in_transit_events,time_in_inter_facility_transit_hours,avg_hours_per_facility,is_express_service,delivery_location_type,num_out_for_delivery_attempts,first_attempt_delivery,total_events_count
0,280267328981,FEDEX_EXPRESS_SAVER,FDXE,20.0,UNKNOWN,Delhi,UNKNOWN,UNKNOWN,Hyderabad,UNKNOWN,...,166.366667,3,11,155.633333,55.455556,True,IN_BOND_OR_CAGE,0,True,16
1,280267329094,FEDEX_EXPRESS_SAVER,FDXE,20.0,UNKNOWN,Delhi,UNKNOWN,UNKNOWN,Noida,UNKNOWN,...,112.533333,3,9,107.550000,37.511111,True,RECEPTIONIST_OR_FRONT_DESK,0,True,18
2,280307632740,FEDEX_EXPRESS_SAVER,FDXE,2.0,UNKNOWN,Mumbai,UNKNOWN,UNKNOWN,Pune,UNKNOWN,...,44.983333,3,7,38.133333,14.994444,True,RESIDENCE,1,True,11
3,280307633276,FEDEX_EXPRESS_SAVER,FDXE,2.0,UNKNOWN,Mumbai,UNKNOWN,UNKNOWN,Pune,UNKNOWN,...,44.983333,3,7,38.166667,14.994444,True,RESIDENCE,1,True,11
4,280439181099,FEDEX_EXPRESS_SAVER,FDXE,32.0,UNKNOWN,Delhi,UNKNOWN,UNKNOWN,Chennai,UNKNOWN,...,191.000000,3,9,180.150000,63.666667,True,RECEPTIONIST_OR_FRONT_DESK,1,True,17
5,280853182067,FEDEX_EXPRESS_SAVER,FDXE,5.0,UNKNOWN,Bangalore,UNKNOWN,UNKNOWN,Chennai,UNKNOWN,...,97.016667,3,8,70.566667,32.338889,True,RESIDENCE,2,False,14
6,280902855329,FEDEX_EXPRESS_SAVER,FDXE,2.0,UNKNOWN,Bangalore,UNKNOWN,UNKNOWN,Delhi,UNKNOWN,...,140.833333,3,11,87.250000,46.944444,True,RESIDENCE,2,False,17
7,280902966660,FEDEX_EXPRESS_SAVER,FDXE,2.5,UNKNOWN,Bangalore,UNKNOWN,UNKNOWN,Hyderabad,UNKNOWN,...,72.883333,3,7,62.266667,24.294444,True,IN_BOND_OR_CAGE,0,True,12
8,280993568461,FEDEX_EXPRESS_SAVER,FDXE,1.5,UNKNOWN,Pune,UNKNOWN,UNKNOWN,Mumbai,UNKNOWN,...,66.183333,3,7,41.383333,22.061111,True,RESIDENCE,2,False,12
9,280998636780,FEDEX_EXPRESS_SAVER,FDXE,3.8,UNKNOWN,Mumbai,UNKNOWN,UNKNOWN,Surat,UNKNOWN,...,95.400000,3,11,70.350000,31.800000,True,RESIDENCE,2,False,16


## Part 6: Output Network Performance Summary CSV
### Objective:
Generate an aggregated summary file "transit_performance_summary.csv"
covering overall, facility, service-type, and delivery performance metrics.

In [36]:
import os
import pandas as pd
import numpy as np
from statistics import mode, StatisticsError

assert 'df_metrics' in globals(), "df_metrics not found — please run Part 3–5 first."

df_summary_base = df_metrics.copy()

# Drop rows with no tracking_number (safety)
df_summary_base = df_summary_base.dropna(subset=["tracking_number"])

# ----------------------------------------------------------
# Overall Metrics
# ----------------------------------------------------------
total_shipments = df_summary_base["tracking_number"].nunique()

transit_hours = df_summary_base["total_transit_hours"].dropna()
avg_transit_hours = round(transit_hours.mean(), 2)
median_transit_hours = round(transit_hours.median(), 2)
std_dev_transit_hours = round(transit_hours.std(ddof=0), 2)
min_transit_hours = round(transit_hours.min(), 2)
max_transit_hours = round(transit_hours.max(), 2)

# ----------------------------------------------------------
# Facility Metrics
# ----------------------------------------------------------
facilities = df_summary_base["num_facilities_visited"].fillna(0)
avg_facilities_per_shipment = round(facilities.mean(), 2)
median_facilities_per_shipment = round(facilities.median(), 2)
try:
    mode_facilities_per_shipment = int(mode(facilities))
except StatisticsError:
    mode_facilities_per_shipment = np.nan

avg_hours_per_facility = round(df_summary_base["avg_hours_per_facility"].dropna().mean(), 2)
median_hours_per_facility = round(df_summary_base["avg_hours_per_facility"].dropna().median(), 2)

# ----------------------------------------------------------
# Service Type Comparison
# ----------------------------------------------------------
service_group = df_summary_base.groupby("service_type", dropna=False)

service_summary = (
    service_group.agg(
        avg_transit_hours_by_service_type=("total_transit_hours", "mean"),
        avg_facilities_by_service_type=("num_facilities_visited", "mean"),
        count_shipments_by_service_type=("tracking_number", "nunique"),
    )
    .round(2)
    .reset_index()
)

# ----------------------------------------------------------
# Delivery Performance
# ----------------------------------------------------------
first_attempt_ratio = (
    df_summary_base["first_attempt_delivery"].mean() * 100
    if "first_attempt_delivery" in df_summary_base.columns else 0
)
avg_out_for_delivery_attempts = (
    df_summary_base["num_out_for_delivery_attempts"].mean()
    if "num_out_for_delivery_attempts" in df_summary_base.columns else 0
)

# ----------------------------------------------------------
# Combine Summary Sections into a Structured Output
# ----------------------------------------------------------

# 1. Overall metrics table
overall_summary = pd.DataFrame([{
    "metric": "Overall Metrics",
    "total_shipments_analyzed": total_shipments,
    "avg_transit_hours": avg_transit_hours,
    "median_transit_hours": median_transit_hours,
    "std_dev_transit_hours": std_dev_transit_hours,
    "min_transit_hours": min_transit_hours,
    "max_transit_hours": max_transit_hours
}])

# 2. Facility metrics table
facility_summary = pd.DataFrame([{
    "metric": "Facility Metrics",
    "avg_facilities_per_shipment": avg_facilities_per_shipment,
    "median_facilities_per_shipment": median_facilities_per_shipment,
    "mode_facilities_per_shipment": mode_facilities_per_shipment,
    "avg_hours_per_facility": avg_hours_per_facility,
    "median_hours_per_facility": median_hours_per_facility
}])

# 3. Delivery performance
delivery_summary = pd.DataFrame([{
    "metric": "Delivery Performance",
    "pct_first_attempt_delivery": round(first_attempt_ratio, 2),
    "avg_out_for_delivery_attempts": round(avg_out_for_delivery_attempts, 2)
}])

### Export All to a Single CSV and show the summary here

In [37]:
# To keep both aggregated tables & grouped service-type comparison
# we'll concatenate them with labels for clarity.
summary_export_path = "C:/Users/Alok verma/Downloads/transit_performance_summary.csv"

with open(summary_export_path, "w", encoding="utf-8", newline="") as f:
    f.write("### OVERALL METRICS ###\n")
overall_summary.to_csv(summary_export_path, index=False, mode="a", encoding="utf-8")

with open(summary_export_path, "a", encoding="utf-8", newline="") as f:
    f.write("\n### FACILITY METRICS ###\n")
    facility_summary.to_csv(f, index=False)
    f.write("\n### SERVICE TYPE COMPARISON ###\n")
    service_summary.to_csv(f, index=False)
    f.write("\n### DELIVERY PERFORMANCE ###\n")
    delivery_summary.to_csv(f, index=False)

print(f"Network performance summary file created successfully!")
print(f"File Path: {os.path.abspath(summary_export_path)}")

# Optional preview of results
display(overall_summary)
display(facility_summary)
display(service_summary)
display(delivery_summary)

Network performance summary file created successfully!
File Path: C:\Users\Alok verma\Downloads\transit_performance_summary.csv


,metric,total_shipments_analyzed,avg_transit_hours,median_transit_hours,std_dev_transit_hours,min_transit_hours,max_transit_hours
0,Overall Metrics,99,94.01,93.25,64.5,15.33,544.28


,metric,avg_facilities_per_shipment,median_facilities_per_shipment,mode_facilities_per_shipment,avg_hours_per_facility,median_hours_per_facility
0,Facility Metrics,2.83,3.0,3,34.04,31.62


,service_type,avg_transit_hours_by_service_type,avg_facilities_by_service_type,count_shipments_by_service_type
0,FEDEX_EXPRESS_SAVER,94.01,2.83,99


,metric,pct_first_attempt_delivery,avg_out_for_delivery_attempts
0,Delivery Performance,84.85,0.97
